Run this overnight to gather a ton of data and make the figures smoother.

In [17]:
addprocs(2) # vary number of concurrent processes here
@everywhere include("../src/TemporalInstanton.jl")
@everywhere include("../src/mat2tmpinst.jl")
@everywhere using TemporalInstanton

 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in eval at ./sysimg.jl:14
 in anonymous at multi.jl:1350
while loading /home/jkersulis/Documents/Dropbox/InstantonAnew/Jonas Edits/Time-Coupled Instanton/temporal-instanton/src/manipulations.jl, in expression starting on line 451
 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in include at ./boot.jl:261
 in include_from_node1 at ./loading.jl:304
 in eval at ./sysimg.jl:14
 in anonymous at multi.jl:1350
while loading /home/jkersulis/Documents/Dropbox/InstantonAnew/Jonas Edits/Time-Coupled Instanton/temporal-instanton/src/manipulations.jl, in expression starting on line 452
 in depwarn at deprecated.jl:73
 in call at deprecated.jl:50
 in include_string at loading.jl:266
 in include_from_node1 at ./lo

In [18]:
# compile everything with this run:
inputData = load_rts96_data(return_as_type=true);
# Thermal model parameters:
inputData.Tamb = 35. # C
inputData.T0 = 60. #46. # initial line steady-state temp

inputData.time_values = 0:30:300 # five minutes in 30-sec steps
inputData.int_length = 300. # seconds = 5 min
Gp,Dp,Rp = inputData.G0,inputData.D0,inputData.R0
inputData.G0 = [0.7*Gp;0.7*Gp;0.7*Gp;0.7*Gp;0.7*Gp;0.7*Gp]
inputData.D0 = [0.9*Dp;0.9*Dp;0.9*Dp;0.9*Dp;0.9*Dp;0.9*Dp]
inputData.R0 = [Rp;1.1*Rp;1.2*Rp;1.3*Rp;1.4*Rp;1.5*Rp]

@time results = solve_temporal_instanton(inputData);

n = length(inputData.k)
nr = length(inputData.Ridx)
T = convert(Int64,length(inputData.G0)/n)
outputData = process_instanton_results(results,n,nr,T,return_as_type=true);

sort(outputData.score)

104-element Array{Float64,1}:
    0.263788
    0.327316
    1.28861 
    2.20415 
    2.32297 
    2.42316 
    4.67876 
    4.86524 
    4.89917 
    5.67827 
    5.93862 
    6.43917 
    6.46089 
    ⋮       
  178.086   
  303.003   
  370.614   
  370.614   
  391.307   
  428.944   
  923.98    
  979.199   
 1319.85    
 1390.03    
 1967.27    
 1967.27    

9.057214 seconds (2.98 M allocations: 143.011 MB, 0.39% gc time)


In [ ]:
using JLD

include("../src/mat2tmpinst.jl")
case = "case2383wp"
mpc = loadcase(case,describe=false)
maxLines = 2
reps = 1
line_times_2383 = Vector{Vector{Float64}}()
num_farms_vec = collect(400:100:2300)

for rep in 1:reps
    gen_count = Vector{Int64}()
    push!(gen_count,length(unique(mpc["gen"][:,1])))

    score_output = Array(Vector{Float64},0)
    sec_elapsed = Vector{Float64}()
    line_times = Vector{Vector{Float64}}()
    bytes_alloc = Vector{Float64}()
    sec_in_gc = Vector{Float64}()
    wind_count = Vector{Int64}()

    for i in 1:length(num_farms_vec)
        num_farms = num_farms_vec[i]
#         println("wind farms: $num_farms")
        push!(wind_count,num_farms)
        penetration = 0.7
        d = mat2tmpinst(case,num_farms,penetration)

        # Thermal model parameters:
        d.Tamb = 35. # C
        d.T0 = 60. #46. # initial line steady-state temp

        d.time_values = 0:600:3600 # one hour in 10-minute steps
        d.int_length = 3600. # seconds = one hour
        
#         d.time_values = 0:30:300 # five minutes in 30-sec steps
#         d.int_length = 300. # seconds = 5 min
        
        Gp,Dp,Rp = (d.G0, d.D0, d.R0)
        d.G0 = [Gp;Gp;Gp;Gp;Gp;Gp]
        d.D0 = [Dp;Dp;Dp;Dp;Dp;Dp]
        d.R0 = [Rp;Rp;Rp;Rp;Rp;Rp]

        n = length(d.k)
        nr = length(d.Ridx)
        T = convert(Int64,length(d.G0)/n)

        timed_results = @timed solve_temporal_instanton(d,maxLines);
        results = timed_results[1]
        o = process_instanton_results(results,n,nr,T,return_as_type=true);
        push!(score_output,o.score)
        push!(sec_elapsed,timed_results[2])
        push!(bytes_alloc,timed_results[3])
        push!(sec_in_gc,timed_results[4])
        push!(line_times,o.linetimes)
    end

    lines_processed = []
    for score_vec in score_output
        push!(lines_processed, sum(score_vec.!=Inf))
    end
    lt = [mean(line_times[i]) for i in 1:length(line_times)]
    push!(line_times_2383, lt)
    save("../data/$(now()).jld","lt",lt)
end

ERROR (unhandled task failure): InterruptException:
 in process_events at ./stream.jl:708
 in wait at ./task.jl:360
 in wait at ./task.jl:286
 in wait at ./channels.jl:93
 in take! at ./channels.jl:82
 in take! at ./multi.jl:792
 in remotecall_fetch at multi.jl:729
 in remotecall_fetch at multi.jl:734
 in anonymous at multi.jl:1485


In [14]:
load("../data/")

In [ ]:
using PyPlot
PyPlot.svg(true)

x = collect(400:100:2300)
y = load("../data/")

In [ ]:
run(`ls ../data/2015*.jld`)